<a href="https://colab.research.google.com/github/iued-uni-heidelberg/cord19/blob/main/cord19WordVectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors

Presentations [slides](intro-word-vectors-DAAD-2021-v22.pdf); Original [slides](word-vectors.pdf)

Word vectors (also known as 'word embeddings') are one of the most popular kinds of AI models. They are extremely useful in many domains. In essence, a word vector is a set of numbers that attempt to capture the meaning of a word. In typical implementations, each word is represented by a set of 200-300 numbers. In linear algebra, a one-dimensional array of numbers is known as a 'vector', hence these sets of numbers representing words' meanings are known as 'word vectors'.

Using neural networks, we can expose the computer to a large amount of text, and allow it to learn an appropriate set of numbers for each word it encounters. In this notebook, we will learn about the most famous of all word vector algorithms, `word2vec`, which was first described by Tomas Mikolov and his team in 2013:

* Tomas Mikolov, Ilya Sutskever, and others, ‘Distributed Representations of Words and Phrases and Their Compositionality’, in Advances in Neural Information Processing Systems 26, ed. by C. J. C. Burges and others (Curran Associates, Inc., 2013), pp. 3111–19 <http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>
* Tomas Mikolov, Kai Chen, and others, ‘Efficient Estimation of Word Representations in Vector Space’, ArXiv:1301.3781 Cs, 2013 <http://arxiv.org/abs/1301.3781>.

In fact, `word2vec` is not a single algorithm, but rather a family of similar algorithms. In this session we will consider just the most famous `word2vec` algorithm, namely the `skip-gram model` trained using `negative sampling`.

## Applications of Word Vectors

Word vectors allow the computer to 'understand' language far more effectively. Rather than seeing each word as simply an arbitrarily different object, a computer using word vectors can analyse each word as a point in 200- or 300-dimenstional space. Words that are similar in meaning will have similar word vectors. And as we will see, the spaces between the word vectors are also significant: the words are arranged in patterns that represent their relationships to one another.

Accordingly, most AI systems that process language now include a word vector layer as part of their architecure. When the system encounters some text (e.g. when you speak to Siri or Alexa), your words are converted into word vectors, *and then* the computer examines what the text says and determines how it should respond.

In the Humanities, word vectors have become a popular modelling tool, because they allow researchers to perform sophisticated analysis on large corpora of text. Some examples include:

* [The Women Writers Vector Toolkit](https://wwp.northeastern.edu/lab/wwvt/index.html)
* William L. Hamilton, Jure Leskovec, and Dan Jurafsky, ‘Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change’, ArXiv:1605.09096 [Cs], 2018 <http://arxiv.org/abs/1605.09096>.
* Ryan Heuser, 'Semantic Networks' <https://ryanheuser.org/word-vectors-4/>

## Training a `word2vec` model in Gensim

It is very easy to train a `word2vec` model in Gensim, which includes Mikolov's original `word2vec` code in its codebase.

In [ ]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data
corpus = api.load('text8')
# api.info()

[==================================================] 100.0% 31.6/31.6MB downloaded


text8 is first 100MB from English wikipedia, 17 million words

The file is downloadable from 
http://mattmahoney.net/dc/text8.zip


In [ ]:
# Examine the corpus to see what is there
api.info("text8")

{'checksum': '68799af40b6bda07dfa47a32612e5364',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'file_name': 'text8.gz',
 'file_size': 33182058,
 'license': 'not found',
 'num_records': 1701,
 'parts': 1,
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'record_format': 'list of str (tokens)'}

In [ ]:
type(corpus)

text8.Dataset

In [ ]:
data = [d for d in corpus]

In [ ]:
len(data)

In [ ]:
len(data[0])

In [ ]:
print(data[0][:100])

In [ ]:
print(data[1][:100])

### Step 1: Set hyperparameters and instantiate model

In [ ]:
vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

model = Word2Vec(
    size=vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

### Step 2: Fit model to corpus

In [ ]:
# build a model
model.build_vocab(corpus)

In [ ]:
# Train the model on the corpus
model.train(sentences=corpus, epochs=5, total_examples=model.corpus_count)

(62533475, 85026035)

### Step 3: Extract word vectors from model

The fully trained model includes all of the weights used to predict the context words for each input word. If you are not planning on training the model further, these weights can be discarded, and you can just keep the weights for the word vectors.

In [ ]:
word_vectors = model.wv
del model # Delete the whole model to free up the computer's RAM

### Step 4: Have a play with the model

There are several ways you can use word vectors. One of the most famous is to use them to compute analogies. The formula is:

<center><em>x</em> is to <em>small</em> as <em>biggest</em> is to <em>big</em></center>

$$x - vector('small') = vector('biggest') - vector('big')$$

$$\therefore x = vector('small') + vector('biggest') - vector('big')$$

In [ ]:
# See the word vector for a particular word
vector = word_vectors['banana']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('university', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('red', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brussels', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('germany', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('strange', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['man'], positive=['king','woman'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['king'], positive=['queen','man'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','girl'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softest'])
print('\n'.join([str(tup) for tup in analogous_words]))

## Step 4A - optional. Using pre-trained models in Gensim

In many applications, you will simply want access to pre-trained word vectors (e.g. for plugging in to another model you are training). If you don't need the vectors to be tailored closely to your particular corpus, then you might like to use some pretrained models.

`word2vec` is not the only word embedding family of algorithms. Another, arguably even more powerful algorithm is the `FastText` algorithm, which Mikolov developed after moving to Facebook:

* Piotr Bojanowski and others, ‘Enriching Word Vectors with Subword Information’, ArXiv:1607.04606, 2017 <http://arxiv.org/abs/1607.04606>.

Instead of computing word vectors for each word, FastText splits each word into its constituent chunks. For example, 'cat' would be split into 'c', 'a', 't', 'ca', 'at' and 'cat', and 'burp' would be split into 'b', 'u', 'r', 'p', 'bu', 'ur', 'rp', 'bur', 'urp' and 'burp'. Then a vector is computer for each chunk that appears in the corpus. Each word is represented as the mean of all the chunks that make it up. FastText is able to learn very good word vectors because it can extract meaning from subword units, e.g. it can see that 'television', 'telegraph' and 'telepathy' all have 'tele' at the front, and can see that 'formality', 'criminality' and 'paucity' share subword units such as 'al' and 'ity'.

You can access many pretrained models using the Gensim downloader. Using the cells below, you can try out some of the different models available through Gensim. Along with `word2vec` and `FastText`, Gensim also supports `Glove` and `Doc2Vec` models.

**NB:** These trained models are very large, and will take a while to download. You may wish to download this notebook and execute the cells below on your own machine, in case Google kicks you out of the Colab environment.

In [ ]:
# See what models are on offer
print(list(api.info()['models'].keys()))

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# 300-dimensional word vectors trained on a huge dataset from Google News
google_news_w2v = api.load('word2vec-google-news-300')

In [ ]:
# x is to Kenya as Canberra is to Australia
google_news_w2v.most_similar(negative=['australia'],positive=['kenya','canberra'], topn=10)

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# Facebook's own FastText vectors, trained on Wikipedia
wikipedia_fasttext = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
# x is to Wharton as London is to Dickens
wikipedia_fasttext.most_similar(negative=['dickens'],positive=['wharton','london'], topn=10)

In [ ]:
# try your own examples... 

## Step 5. Training the Armenian and Georgian models

Now let's train a model on our own corpus. You can try to create Armenian / Ukrainian / Russian / German / French word vector model.

Armenian, Ukrainian, Russian plaintext wikipedias are available at:
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2735#

We can use the same size ~15 Million Words, (200MB) of these corpora to make sure the model is built within a reasonable time; however, when you have more time, you can try a more complete corpus.

It is possible to download/unzip/upload the files manually, however, it is faster to automatically load them directly from Heidelberg server space https://heibox.uni-heidelberg.de/d/42a07c9e95774e099a81/  into your colab environment. To do this run the cells 5a or 5b, or 5c, or 5d, or 5e -- depending on which language you prefer -- to download the data. Then skip other language and continue with Step5 -- building the model for your preferred language from the data you downloaded.


In [ ]:
!rm /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt
!rm index.html?dl=1

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt': No such file or directory
rm: cannot remove 'index.html?dl=1': No such file or directory


### Either: 5a Armenian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ffe527ed3d1e4b4cb43a/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5b Ukrainian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/676bef3a6db8482e9665/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5c German

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ca0a085347c34563a6e6/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5d French

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/5cdb485efd4046f2a457/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5e Russian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/d8d381152ceb479080ca/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5f Georgian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/fa3509d869b949459f91/?dl=1

Lemmatized corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/b381c458ad5e4773a77d/?dl=1

--2021-09-03 08:53:16--  https://heibox.uni-heidelberg.de/f/b381c458ad5e4773a77d/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/7b521159-dc90-4e0f-89fa-6876b167b9fb/ka_lem.txt [following]
--2021-09-03 08:53:16--  https://heibox.uni-heidelberg.de/seafhttp/files/7b521159-dc90-4e0f-89fa-6876b167b9fb/ka_lem.txt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 329202500 (314M) [text/plain]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 313.95M  14.9MB/s    in 21s     

2021-09-03 08:53:37 (15.2 MB/s) - ‘index.html?dl=1’ saved [329202500/329202500]



In [ ]:
!cp index.html?dl=1 wiki_geo_lem.txt

In [ ]:
!head --lines=10 wiki_geo_lem.txt

ედუარდი
შევარდნაძე
ედუარდი
ამბროსის
ძე
შევარდნაძე
(
დ
.
25


In [ ]:
FIn = open("wiki_geo_lem.txt", 'r')
FOut = open('wiki_geo_l.txt', 'w')

i = 0
for SLine in FIn:
  i = i + 1
  SLine = SLine.strip() + ' '
  if i % 10000 == 0:
    SLine += '\n'
  
  FOut.write(SLine)

FOut.flush()
FOut.close()


In [ ]:
!head --lines=10 wiki_geo_l.txt

In [ ]:
!wc wiki_geo_lem.txt

 18054750  18054749 329202500 wiki_geo_lem.txt


In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

Alternatively, copy lemmatized corpus

In [ ]:
!cp wiki_geo_l.txt /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

## or: CORD corpus models
modify this to merge more files into your corpus 

In [49]:
!rm /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/e9643d5305d94c5cb787/?dl=1
!mv index.html?dl=1 file1.zip
!wget https://heibox.uni-heidelberg.de/f/76525e4278114c438f1c/?dl=1
!mv index.html?dl=1 file2.zip
!wget https://heibox.uni-heidelberg.de/f/062f72f9e79e4ba5ae4e/?dl=1
!mv index.html?dl=1 file3.zip
!wget https://heibox.uni-heidelberg.de/f/4e290ad989a046caaef4/?dl=1
!mv index.html?dl=1 file4.zip
!wget https://heibox.uni-heidelberg.de/f/5ec055c8956348d98a44/?dl=1
!mv index.html?dl=1 file5.zip

!unzip file1.zip
!unzip file2.zip
!unzip file3.zip
!unzip file4.zip
!unzip file5.zip

In [ ]:
!rm file*.zip
!cat covid1_lem.txt covid2_lem.txt covid3_lem.txt covid4_lem.txt covid5_lem.txt >covid_lem.txt
!cp covid_lem.txt /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

!head --lines=10 covid_lem.txt
!wc covid_lem.txt
!rm covid_lem.txt



### Stage6: Training own model

optional: to clean disk space, we remove the downloaded file

In [ ]:
# optional: to clean disk space, we remove the downloaded file
!rm index.html\?dl\=1

In [7]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('myOwnLangText8.txt')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [8]:
import gensim.models
corpusOwn = MyCorpus()

optional: examining what is in the corpus after standard preprocessing

In [9]:
# Optional: Examining our corpus format
type(corpusOwn)

__main__.MyCorpus

In [10]:
dataOwn = [d for d in corpusOwn]

In [ ]:
print(len(dataOwn))
print(dataOwn[0])
print(len(dataOwn[0]))
print(dataOwn[4])
print(len(dataOwn[4]))
print(dataOwn[5])
print(len(dataOwn[5]))

... Initialising global parameters for our modelL vector size, collocation window, skip-grams, negative sampling....

In [12]:
del dataOwn

In [13]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data

In [33]:
vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

modelOwn = Word2Vec(
    size=vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

... this cell may run for ~2 min or so...

In [34]:
modelOwn.build_vocab(corpusOwn)

THIS MAY TAKE LONG!!! ... training the model may take 9 to 15 minutes... (just grab a cup of coffee or a sandwich while you are waiting... You may try chaning the number of epochs; if the number is lower the training is faster, but the quality may be lower...

5 epochs = 50 min
8 epochs = 1 h 17 min (77 min)
10 epochs ~? 100 min ~ 1h 40 min?

In [35]:
modelOwn.train(sentences=corpusOwn, epochs=10, total_examples=modelOwn.corpus_count)

(524576887, 688888256)

Now we copy word vectors and remove the model from memory (just to free up the resources...)

In [36]:
word_vectors_own = modelOwn.wv

In [18]:
del modelOwn

In [37]:
vector = word_vectors_own['however']
print(vector)

[ 0.05926263 -0.15667033  0.03318803 -0.18686087 -0.01929474  0.0850039
 -0.45187265 -0.06674725 -0.21036066  0.15188602 -0.11955456 -0.24665129
 -0.17086191  0.1575264   0.1435385   0.05060921  0.13252607  0.05781075
 -0.2439971  -0.21926726  0.17216878  0.08300912  0.01004632  0.07455021
  0.1758465   0.09111493  0.27284402 -0.05395681 -0.00918299 -0.18407369
 -0.17696461  0.2809646  -0.14535953 -0.09520941  0.05423077  0.32214114
  0.15689044 -0.01155012  0.22412404  0.17828822  0.23961754 -0.1286369
 -0.03883589  0.1639629  -0.14529589 -0.40622824  0.12130663  0.21533312
 -0.05149914  0.07464544  0.02623041  0.20257315  0.02297816 -0.04642959
 -0.30257323  0.10443529  0.10155771 -0.04316276  0.06895683 -0.15213935
  0.02633966  0.19673464 -0.10915372  0.24391913 -0.34283364 -0.33969817
 -0.09345365  0.06161407  0.2184848   0.18220435  0.00633711 -0.20755926
 -0.09029377  0.31810436  0.17724325 -0.04032632  0.13063999  0.10723896
  0.06444924  0.22883163  0.11574932  0.0286354   0.3

In [48]:
similar_words = word_vectors_own.most_similar('however', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

('although', 0.9558818340301514)
('nevertheless', 0.944508969783783)
('nonetheless', 0.9050902128219604)
('moreover', 0.861580491065979)
('furthermore', 0.8573136925697327)
('unfortunately', 0.8478303551673889)
('though', 0.8315995335578918)
('indeed', 0.8223739266395569)
('while', 0.8093974590301514)
('still', 0.805526614189148)
('surprisingly', 0.8004773855209351)
('but', 0.7992550134658813)
('additionally', 0.7867470979690552)
('despite', 0.7771902084350586)
('therefore', 0.7697354555130005)
('likewise', 0.7666776776313782)
('besides', 0.7603017091751099)
('importantly', 0.7583843469619751)
('notably', 0.7536568641662598)
('thirdly', 0.7478072047233582)
('notwithstanding', 0.7448851466178894)
('fortunately', 0.7411450147628784)
('lastly', 0.7401111721992493)
('meanwhile', 0.7368455529212952)
('interestingly', 0.736506998538971)
('because', 0.7339373230934143)
('thus', 0.7322976589202881)
('since', 0.7287991046905518)
('whilst', 0.7268706560134888)
('yet', 0.7228360772132874)


Now we can examine the output of our word embeddings

### Georgian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['სამყარო']
print(vector)

[-0.62176156  0.23949477 -0.5179568   1.001093   -0.12505919  0.23363337
  0.36481667 -0.33915916 -0.06151456 -0.15845872  0.1443672   0.35141632
  0.41165924  0.5007546   0.04449251  0.6967667  -0.27176398  0.05205816
  0.27029327  0.20663835 -0.36598527 -0.14222133  0.16173883 -0.08235938
 -0.5496693  -0.15209736  0.48121738 -0.15404233  0.8891668   0.76310915
 -0.3202305  -0.18489017  0.3939411   0.15373786  0.02998169  0.20106426
 -0.0862966  -0.50034994 -0.05873515 -0.24013674  0.12750806  0.17326556
 -0.29788643  0.68244714 -0.21748939 -0.42120582  0.13775422 -0.35998118
  0.36402082  0.10524762 -0.0595954   0.3082781  -0.263306   -0.60933733
 -0.21795647 -0.04971887  0.09330089  0.01892476 -0.49159762 -0.11128791
 -0.30394047 -0.82244855 -0.48691386 -0.17550847 -0.15397266 -0.08480422
 -0.20583957 -0.67179024  0.77594376 -0.4534941   0.0747933   0.30824754
 -0.17382185 -0.47079867 -0.27730626  0.18782099 -0.10516048  0.51495117
  0.23847543 -0.11177064  0.04745216  0.10895227 -0

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('არსება', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

('არსების', 0.8350973129272461)
('არსებები', 0.8056425452232361)
('დემონები', 0.8028587102890015)
('არსებებს', 0.8012523651123047)
('არსებებად', 0.7943331003189087)
('არსებად', 0.789718747138977)
('არსებანი', 0.7854664325714111)
('არსებაა', 0.7757723331451416)
('გეა', 0.7722355723381042)
('ყოვლისშემძლე', 0.7706775665283203)
('არსებების', 0.7645055055618286)
('სულები', 0.7611775994300842)
('მოკვდავი', 0.7600510120391846)
('დემონებზე', 0.754485011100769)
('ღმერთებზე', 0.7539801597595215)
('ლოპარების', 0.7538608312606812)
('კარმას', 0.7527127265930176)
('არსებათა', 0.7514225840568542)
('ჰუმანოიდური', 0.7510616183280945)
('სხეულისგან', 0.747897744178772)
('ადამიან', 0.7469550371170044)
('ღმერთები', 0.7469128370285034)
('არსებაზე', 0.7454417943954468)
('ღმერთია', 0.7444026470184326)
('სულებისა', 0.7416738271713257)
('მოგიზგიზე', 0.7412869334220886)
('სილამაზესა', 0.7398790121078491)
('ობობის', 0.7397568821907043)
('მსგავსს', 0.7365857362747192)
('მრავალთავიანი', 0.736303448677063)


In [ ]:
# blue
similar_words = word_vectors_own.most_similar('ლურჯი', topn=30) 
print('\n'.join([str(tup) for tup in similar_words]))

('თეთრი', 0.8326250910758972)
('მწვანე', 0.8310437202453613)
('ფერი', 0.8173065185546875)
('ყვითელი', 0.8154611587524414)
('წითელი', 0.8072329759597778)
('ოქროსფერი', 0.8043473958969116)
('ცისფერ', 0.8018262386322021)
('ვარდისფერი', 0.7977309823036194)
('ოქროსფერ', 0.7818973660469055)
('ფერებია', 0.7646973133087158)
('იასამნისფერი', 0.7630882263183594)
('ვერცხლისფერი', 0.7574926018714905)
('მუქი', 0.754925549030304)
('ცისფერი', 0.7541515827178955)
('სტაფილოსფერი', 0.7397026419639587)
('თეთრ', 0.7386224269866943)
('ფერებშია', 0.7350391745567322)
('ნარინჯისფერი', 0.7323054671287537)
('ფერებად', 0.7321016788482666)
('წითელ', 0.7310094833374023)
('შორტები', 0.7249136567115784)
('იისფერი', 0.7244935035705566)
('ვერცხლისფერ', 0.7243334054946899)
('ფერისაა', 0.7218009233474731)
('ფერით', 0.7189768552780151)
('ყავისფერი', 0.7142767906188965)
('ნახევარმთვარე', 0.7126744389533997)
('მოწითალო', 0.7084356546401978)
('იისფერ', 0.7074872255325317)
('ნარინჯისფერ', 0.7070273160934448)


In [ ]:
# france
similar_words = word_vectors_own.most_similar('საფრანგეთი', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['კაცი'], positive=['მეფე','ქალი'])
print('\n'.join([str(tup) for tup in analogous_words]))

('დედოფალი', 0.7610162496566772)
('ფარაონის', 0.6152430772781372)
('მეფეების', 0.6116519570350647)
('სამეფო', 0.6032817363739014)
('დედოფლად', 0.5937286615371704)
('ამენჰოტეპ', 0.5872259140014648)
('სეთი', 0.5852410197257996)
('მეფეა', 0.577671229839325)
('მემკვიდრის', 0.5736474990844727)
('სტიუარტების', 0.5730718970298767)


In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′) 
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′) 
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′) 
analogous_words = word_vectors_own.most_similar(negative=['პირი'], positive=['ხალხი','ქალიშვილი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])

### Armenian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['աշխարհը']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('աշխարհը', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('կապույտ', topn=10) 
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('ֆրանսիա', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['մարդ'], positive=['թագավոր','կին'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′) 
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′) 
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′) 
analogous_words = word_vectors_own.most_similar(negative=['մեծ'], positive=['ամենամեծը','փոքր'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])